# Problem description
The company BIM (Best International Machines) produces two types of microchips, logic chips (1g silicon, 1g plastic, 4g copper) and memory chips (1g germanium, 1g plastic, 2g copper). Each of the logic chips can be sold for a 12€ profit, and each of the memory chips for a 9€ profit. The current stock of raw materials is as follows: 1000g silicon, 1500g germanium, 1750g plastic, 4800g copper. How many microchips of each type should be produced to maximize profit while respecting the availability of raw material stock?

# Linear Programming Formulation

## Decision Variables
Let:
- $x_1$: Number of **logic chips** to be produced.
- $x_2$: Number of **memory chips** to be produced.

## Objective Function
Maximize the total profit:
$$
\text{Maximize: } Z = 12x_1 + 9x_2
$$ 

## Constraints
1. Production of logic chips:
   $$
   x_1 \leq 1000
   $$
2. Production of memory chips:
   $$
   x_2 \leq 1500
   $$
3. Combined production limit:
   $$
   x_1 + x_2 \leq 1750
   $$
4. Resource availability (silicon usage):
   $$
   4x_1 + 2x_2 \leq 4800
   $$
5. Non-negativity constraints:
   $$
   x_1 \geq 0, \; x_2 \geq 0
   $$


In [109]:
import pyomo.environ as pyo
solver = "appsi_highs"
SOLVER = pyo.SolverFactory(solver)

## Problem specific

In [112]:
model = pyo.ConcreteModel("BIM production planning:version 1")

# decision variables
model.x1 = pyo.Var(domain = pyo.NonNegativeReals)
model.x2 = pyo.Var(domain = pyo.NonNegativeReals)

# objective function
model.profit = pyo.Objective(expr= 12*model.x1 + 9*model.x2, sense = pyo.maximize)

# constraints
model.silicon = pyo.Constraint(expr = model.x1 <= 1000)
model.germanium = pyo.Constraint(expr = model.x2 <= 1500)
model.plastic = pyo.Constraint(expr = model.x1 + model.x2 <= 1750)
model.copper = pyo.Constraint(expr = 4* model.x1 + 2* model.x2 <= 4800)

# optimize and solve
SOLVER.solve(model)

print(f"x = ({model.x1.value:.1f}, {model.x2.value:.1f})")
print(f"optimal value = {pyo.value(model.profit):.1f}")

x = (650.0, 1100.0)
optimal value = 17700.0


# Data Driven Model

In [115]:
# data
products = {
    'logic chip':{'price':12},
    'memory chip':{'price':9}
        }
resources ={
    'silicon':{'available':1000},
    'germanium':{'available':1500},
    'plastic':{'available':1750},
    'copper':{'available':4800},
}
processes = {
    'logic chip':{'silicon':1, 'plastic':1, 'copper':4},
    'memory chip': {'germanium':1,'plastic':1, 'copper':2}
}

In [117]:
model = pyo.ConcreteModel('BIM production plannning: version 2')

model.PRODUCTS = pyo.Set(initialize = products.keys())
model.RESOURCES = pyo.Set(initialize = resources.keys())

# Parameters
model.Price = pyo.Param(model.PRODUCTS, initialize=lambda model, p: products[p]['price'])
model.Availability = pyo.Param(model.RESOURCES, initialize=lambda model, r: resources[r]['available'])
model.Requirements = pyo.Param(model.PRODUCTS, model.RESOURCES, 
                                initialize=lambda model, p, r: processes[p].get(r, 0))  # Default to 0 if not in process

# decision variables
model.x = pyo.Var(model.PRODUCTS, domain = pyo.NonNegativeReals)

# objective funtion
@model.Objective(sense=pyo.maximize)
def profit(model):
    return pyo.quicksum(
        model.Price[product] * model.x[product] for product in model.x
    )

@model.Constraint(model.RESOURCES)
def ResourceConstraint(model, resource):
    return pyo.quicksum(
        model.Requirements[product, resource]*model.x[product] for product in model.x
    ) <= model.Availability[resource]

result = SOLVER.solve(model)

# Display Results
for product in model.PRODUCTS:
    print(f"Produce {model.x[product]():.2f} units of {product}")
print(f"Maximum Profit: {model.profit():.2f}")

Produce 650.00 units of logic chip
Produce 1100.00 units of memory chip
Maximum Profit: 17700.00


# Using OOPs concepts

In [45]:
import pyomo.environ as pyo

class ProductionModel:
    def __init__(self, products, resources, processes):
        super().__init__()
        self.products = products
        self.resources = resources
        self.processes = processes
        self.pyomo_model = pyo.ConcreteModel()  # Renamed to avoid conflict
        self.solved = False

    def build_model(self):
        model = self.pyomo_model
        
        model.PRODUCTS = pyo.Set(initialize=self.products.keys())
        model.RESOURCES = pyo.Set(initialize=self.resources.keys())

        model.x = pyo.Var(model.PRODUCTS, domain=pyo.NonNegativeReals)
        
        # Objective function
        @model.Objective(sense=pyo.maximize)
        def profit(model):
            return sum(
                self.products[product]['price'] * model.x[product] for product in model.PRODUCTS
            )
        
        # Constraints
        @model.Constraint(model.RESOURCES)
        def ResourceConstraint(model, resource):
            return sum(
                self.processes[product].get(resource, 0) * model.x[product] for product in model.PRODUCTS
            ) <= self.resources[resource]['available']
        
    def solve(self, solver="glpk"):
        self.build_model()
        solver_instance = pyo.SolverFactory(solver)
        solver_instance.solve(self.pyomo_model)
        self.solved = True

    def report(self):
        if not self.solved:
            self.solve()
        print(f"Profit = {pyo.value(self.pyomo_model.profit)}")
        print("\nProduction Report")
        for product in self.pyomo_model.PRODUCTS:
            print(f"{product} produced = {pyo.value(self.pyomo_model.x[product])}")

model = ProductionModel(products, resources, processes)
model.solve()
model.report()

Profit = 17700.0

Production Report
logic chip produced = 650.0
memory chip produced = 1100.0


# Canonical form

In [68]:
import numpy as np

model = pyo.ConcreteModel('BIM production planning in matric form')

# define the number of variavles and constriants
n_vars = 2
n_constraints = 4

# decision varaibles
model.x = pyo.Var(range(n_vars), domain = pyo.NonNegativeReals)

# define data
c = np.array([12, 9])
A = np.array([[1,0],[0,1],[1,1],[4,2]])
b = np.array([1000,1500,1750,4800])

# Objective function
model.profit = pyo.Objective(
    expr=sum(c[i] * model.x[i] for i in range(n_vars)), sense=pyo.maximize
)
# constraints
model.Constraints = pyo.ConstraintList()
for i in range(n_constraints):
    model.Constraints.add(expr=sum(A[i,j] * model.x[j] for j in range(n_vars)) <= b[i])

SOLVER.solve(model)
optimal_x = [pyo.value(model.x[i]) for i in range(n_vars)]
print(f"x = {tuple(np.round(optimal_x, 1))}")
print(f"optimal value = {pyo.value(model.profit):.1f}")

x = (650.0, 1100.0)
optimal value = 17700.0


# using Pulp

In [146]:
# problem specific
model = LpProblem(name="BIM_Production_Planning_version_5", sense=LpMaximize)

# Define the decision variables
x1 = LpVariable(name="x1_logic_chip", lowBound=0)  # Logic chips
x2 = LpVariable(name="x2_memory_chip", lowBound=0)  # Memory chips

# Define the objective function
model += 12 * x1 + 9 * x2, "Maximize_Profit"

# Add the constraints
model += x1 <= 1000, "Silicon_Constraint"
model += x2 <= 1500, "Germanium_Constraint"
model += x1 + x2 <= 1750, "Plastic_Constraint"
model += 4 * x1 + 2 * x2 <= 4800, "Copper_Constraint"

# Solve the problem
model.solve()

# Step 6: Display the results
print("Optimal Production Plan:")
print(f"Produce {x1.varValue:.2f} units of logic chips (x1)")
print(f"Produce {x2.varValue:.2f} units of memory chips (x2)")
print(f"Maximum Profit: {model.objective.value():.2f}")


Optimal Production Plan:
Produce 650.00 units of logic chips (x1)
Produce 1100.00 units of memory chips (x2)
Maximum Profit: 17700.00


In [148]:
from pulp import *
model = LpProblem('BIM_Production_Planning_version_6', sense = LpMaximize)

# define decision variables
x = {product: LpVariable(name = f"x_{product}", lowBound = 0) for product in products.keys()}

# Add Objective Function
model += lpSum(products[product]['price']* x[product] for product in products.keys()),'Total_Profit'

# Add Constraints
for resource, data in resources.items():
    model += lpSum(processes[product].get(resource, 0) * x[product] for product in products.keys()) <= data['available'], f"Resource_Const_{resource}"

if model.solve():
    print('model_solved')
else:
    print('There is some problem')
for product in products.keys():
    print(f"Product {x[product].varValue} units of {product} should be produced")
print(f"Maximum Profit: {model.objective.value()}")

model_solved
Product 650.0 units of logic chip should be produced
Product 1100.0 units of memory chip should be produced
Maximum Profit: 17700.0


# Using GurobiPy

In [151]:
from gurobipy import *

model = Model('BIM_Production_Planning_version_7')

x = model.addVar(vtype=GRB.CONTINUOUS, name='Logic_chip', lb=0)
y = model.addVar(vtype = GRB.CONTINUOUS, name='Memory_chip', lb=0)

model.setObjective(12*x + 9*y, GRB.MAXIMIZE)

model.addConstr(x <= 1000, name='silicon_avail')
model.addConstr(y <= 1500, name='germanium_avail')
model.addConstr(x+y <= 1750, name='plastic_avial')
model.addConstr(4*x +2*y <= 4800, name='copper_avail')

model.update()
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal Production Plan:")
    print(f"Produce {x.x:.2f} units of logic chips (x)")
    print(f"Produce {y.x:.2f} units of memory chips (y)")
    print(f"Maximum Profit: {model.objVal:.2f}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-26
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xc748ad9b
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [9e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 5e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+04   3.250000e+02   0.000000e+00      0s
       2    1.7700000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.770000000e+04
Optimal Production Plan:
Produ

# Generalise 

In [159]:
model = Model('BIM_Production_Planning_version_2')

# sets
product_set = list(products.keys())
resource_set = list(resources.keys())

# decision variables
x = model.addVars(product_set, vtype = GRB.CONTINUOUS, lb = 0, name = 'x')

# objectve funtion
model.setObjective(quicksum(products[product]['price'] * x[product] for product in product_set), GRB.MAXIMIZE)

# constraints
model.addConstrs((quicksum(processes[product].get(resource, 0) * x[product] for product in product_set) 
                 <= resources[resource]['available'] for resource in resource_set), name = "Resource_Avail")

model.update()
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Production Plan:")
    for p in product_set:
        print(f"Produce {x[p].x:.2f} units of {p}")
    print(f"Maximum Profit: {model.objVal:.2f}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xc748ad9b
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [9e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 5e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+04   3.250000e+02   0.000000e+00      0s
       2    1.7700000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.770000000e+04

Optimal Production Plan:
Produce 650.00 units of logic chip
Produce 1100.00 units of memory chip
Maximum Profit: 17700.0

### Thank you